In [ ]:
匹配账户开户信息表，补充户名

In [3]:
import pandas as pd
all_data = pd.read_csv("E:\\7.3徐州网赌\\流水合并剔除冲正.csv",low_memory = False)
col_name = all_data.columns[1:33]
df1 = all_data
df2 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
dfs = pd.merge(df1,df2, how ='left',left_on='交易卡号',right_on='银行卡号')
print(dfs.columns)

Index(['index', '交易卡号', '交易账号', '查询反馈结果原因', '交易户名', '交易证件号码', '交易时间', '交易金额',
       '交易余额', '收付标志', '交易对手账卡号', '现金标志', '对手户名', '对手身份证号', '对手开户银行', '摘要说明',
       '交易币种', '交易网点名称', '交易场所', '交易发生地', '交易是否成功', '传票号', 'IP地址', 'MAC地址',
       '对手交易余额', '交易流水号', '渠道', '日志号', '凭证种类', '凭证号', '交易柜员号', '备注', '交易类型',
       'path', '序号', '姓名', '银行卡号'],
      dtype='object')


In [4]:
import numpy as np
df_fill = dfs.applymap(lambda x: np.NaN if str(x)=='' else x)
df_fill['交易卡号'][df_fill['交易卡号'].isnull()]=df_fill['交易账号'][df_fill['交易卡号'].isnull()]
print(df_fill.groupby('交易卡号').size())
df_fill['交易户名'] = df_fill['姓名'].str.strip()
df_fill = df_fill.iloc[:,1:34]
print(df_fill)

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


交易卡号
1001810033940469_C100CNY      13
1001810044484698_C100CNY     211
1001810069314478_C100CNY    1000
1001810073566360_C100CNY    2000
1001810074033042_C100CNY    1000
                            ... 
9070611100100100279932         1
95040078801200000557           1
95040078801900000558           1
9558801001151426019          512
9559982021721381410         1318
Length: 1627, dtype: int64
                      交易卡号                  交易账号 查询反馈结果原因 交易户名  交易证件号码  \
0         6216911507196744  50000000001025453271       成功   许芳     NaN   
1         6216911507196744  50000000001025453271       成功   许芳     NaN   
2         6216911507196744  50000000001025453271       成功   许芳     NaN   
3         6216911507196744  50000000001025453271       成功   许芳     NaN   
4         6216911507196744  50000000001025453271       成功   许芳     NaN   
...                    ...                   ...      ...  ...     ...   
2019421  12081101040051326                   NaN      NaN  NaN     NaN   
2019422  1208

In [16]:
# df = df_fill[df_fill['交易卡号']=='6216911507196744']
# df_余额 = pd.DataFrame(columns = ['交易卡号','交易户名','交易时间','交易余额'])
# df_余额.loc[0] = df[['交易卡号','交易户名','交易时间','交易余额']].loc[df.index.max()]
# df_余额

,交易卡号,交易户名,交易时间,交易余额
0,6216911507196744,许芳,2020-03-27 14:37:24,144030.76


In [18]:
df = df_fill
df_余额 = pd.DataFrame(columns = ['交易卡号','交易户名','交易时间','交易余额'])
df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = col_name,keep = 'last').value_counts()
    if len(bool_S)==1:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':bool_S[1]}
    df_余额.loc[i] = df_every[['交易卡号','交易户名','交易时间','交易余额']].loc[df_every.index.max()]

df_S2.to_excel("E:\\7.3徐州网赌\\单卡去重统计.xlsx",index = None)
print(df.duplicated(subset = col_name).value_counts())
df_余额.to_excel("E:\\7.3徐州网赌\\单卡最后一笔交易余额.xlsx",index = None)
print(df_S1)
print(df_S2)

False    1628251
True      391175
dtype: int64
                          交易卡号     0
0     1001810033940469_C100CNY    13
1     1001810044484698_C100CNY   211
2     1001810069314478_C100CNY  1000
3     1001810073566360_C100CNY  2000
4     1001810074033042_C100CNY  1000
...                        ...   ...
1622    9070611100100100279932     1
1623      95040078801200000557     1
1624      95040078801900000558     1
1625       9558801001151426019   512
1626       9559982021721381410  1318

[1627 rows x 2 columns]
                          交易卡号  Size False True
0     1001810033940469_C100CNY    13     7    6
1     1001810044484698_C100CNY   211   211    0
2     1001810069314478_C100CNY  1000  1000    0
3     1001810073566360_C100CNY  2000  1001  999
4     1001810074033042_C100CNY  1000  1000    0
...                        ...   ...   ...  ...
1622    9070611100100100279932     1     1    0
1623      95040078801200000557     1     1    0
1624      95040078801900000558     1     1    0
1625

In [19]:
dup_data = df.drop_duplicates(subset = col_name,keep = 'last')
dup_data.to_csv("E:\\7.3徐州网赌\\流水户名.csv",index = None)
print(df.duplicated(subset = col_name).value_counts())


False    1628251
True      391175
dtype: int64
